In [0]:
from google.colab import files
import numpy as np
import pandas as pd
from scipy import stats
import os,sys
import scipy.sparse as sparse
from scipy.sparse.linalg import spsolve

In [2]:
#To get access to the files on google drive an authorization code it nessacery which will be given by the linked below
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [3]:
input_data_path="/gdrive/My Drive/Colab Notebooks/Input_files/produced"
output_data_path="/gdrive/My Drive/Colab Notebooks/Output_files"
os.listdir(input_data_path)

['data_InstaCart_rating.csv',
 'product_class.csv',
 'user_class_products.csv',
 'data_InstaCart_sparse.npz',
 'user_class.csv',
 'data_InstaCart_sparse_over100.npz',
 'over600',
 'data_InstaCart_sampled.csv',
 'data_InstaCart_user_triple.csv',
 'data_InstaCart_user_double.csv',
 'data_InstaCart_data_count.csv',
 'data_InstaCart_data_count_depart_aisle.csv',
 'data_InstaCart_train.csv',
 'data_InstaCart_order_product.csv',
 'data_InstaCart.csv']

In [0]:
data=pd.read_csv(os.path.join(input_data_path,'data_InstaCart_rating.csv'))

In [29]:
data.head(20)

,user_id,product_id,rating
0,1,196.0,100.0
1,1,10258.0,91.0
2,1,10326.0,9.0
3,1,12427.0,91.0
4,1,13032.0,36.0
5,1,13176.0,18.0
6,1,14084.0,9.0
7,1,17122.0,9.0
8,1,25133.0,82.0
9,1,26088.0,27.0


In [49]:
data.loc[data.user_id==1,['product_id','rating']]

,product_id,rating
0,196.0,100.0
1,10258.0,91.0
2,10326.0,9.0
3,12427.0,91.0
4,13032.0,36.0
5,13176.0,18.0
6,14084.0,9.0
7,17122.0,9.0
8,25133.0,82.0
9,26088.0,27.0


In [6]:
customers = list(np.sort(data.user_id.unique())) # Get our unique customers
products = list(data.product_id.unique()) # Get our unique products that were purchased
rating = list(data.rating) # All of our purchases

rows = data.user_id.astype('category', categories = customers).cat.codes 
# Get the associated row indices
cols = data.product_id.astype('category', categories = products).cat.codes 
# Get the associated column indices
purchases_sparse = sparse.csr_matrix((rating, (rows, cols)), shape=(len(customers), len(products)))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: FutureWarning: specifying 'categories' or 'ordered' in .astype() is deprecated; pass a CategoricalDtype instead
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: FutureWarning: specifying 'categories' or 'ordered' in .astype() is deprecated; pass a CategoricalDtype instead
  import sys


In [7]:
purchases_sparse

<206209x49685 sparse matrix of type '<class 'numpy.float64'>'
	with 13863746 stored elements in Compressed Sparse Row format>

In [8]:
np.shape(purchases_sparse)

(206209, 49685)

In [9]:
matrix_size = purchases_sparse.shape[0]*purchases_sparse.shape[1] # Number of possible interactions in the matrix
num_purchases = len(purchases_sparse.nonzero()[0]) # Number of items interacted with
sparsity = 100*(1 - (num_purchases/matrix_size))
sparsity

99.86468445760907

In [28]:
print(purchases_sparse[0:2,0:15].todense())

[[100.  91.   9.  91.  36.  18.   9.   9.  82.  27.  27.   9.   9.   9.
   18.]
 [  0.   0.   0.   0.   0.  10.   0.   0.   0.   0.   0.   0.   0.   0.
    0.]]


In [0]:
sparse.save_npz(os.path.join(input_data_path,'data_InstaCart_sparse.npz'), purchases_sparse)